In [4]:
pip3 install pandas

SyntaxError: invalid syntax (<ipython-input-4-83358da30272>, line 1)

In [3]:
import pandas as pd
import numpy as np
import cv2
import os
import imutils
from keras.applications import VGG19, VGG16
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from PIL import Image
import keras
from keras import Sequential, losses, optimizers, Input
from keras.layers import Dense, Conv2D, MaxPool2D, Average, Flatten, Dropout, Activation
from keras.utils import to_categorical, plot_model,vis_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array
from skimage import io
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pywt
from scipy import signal


ModuleNotFoundError: No module named 'pandas'

In [ ]:
class DataModel:
    """
            DataModel
            not used currently in the program.
    """
    def __init__(self):
        self.items_labels = None
        self.images = []
        self.curated_data = {}
    def add_image(self, image_array):
        self.images.append(image_array)
    def add_labels(self, df):
        self.item_labels = df
    def add_curated_data(self, image, emotion):
        self.curated_data['segment'] = image #Array from csv of each segment
        self.curated_data['label'] = label #Label
        self.curated_data['scalogram'] =scaledImage #scalogram of each segment

def load_data(number_of_items=90):
    """
        number_of_items -> Number of items to return
        returns the data in a dictionary of images and labels.
    """
    path = "butterworth_kalman_result_Scalogram"
    path1= "butterworth_kalman_result_Scalogram/Cropped"
    data = [] 
    curated_data = {"label":[], "scalogram":[]}
    for subject_name in os.listdir(path)[:number_of_items]:
        # At the start of the iteration build a data model
        data_model = DataModel()
        if subject_name == ".DS_Store":
            continue
        if subject_name  ==".ipynb_checkpoints":
            continue
        if subject_name  ==".svn":
            continue
        print ("Going through subject:" + subject_name)
        base=os.path.basename(path+"/"+subject_name)
        labelData=os.path.splitext(base)[0]
        print(labelData)
        i=0
        for items in os.listdir(path+"/"+subject_name):
            if items == ".DS_Store":
                continue
            if items.endswith(".png"):
                #i=i+1
                #print(i)
                try:
                    im2 = cv2.imread(path+"/"+subject_name+"/"+items)
                    #plt.imshow(im2)
                    #plt.show()
                    crop_img = im2[30:20+235, 50:50+342]
                    im = cv2.resize(crop_img, (224,224)) # Changing into 80x80X3
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                    #print(type(crop_img))
                    #plt.imshow(crop_img)
                    #plt.show()
                    

              #curated_data['segments'].append(df1)
                    #curated_data['scalogram'].append(crop_img)
                    curated_data['scalogram'].append(im)
                    curated_data['label'].append(labelData)
              #print(df1)
                except:
                      df = None  
        #data.append(data_model) # Save all the data

    return curated_data


In [ ]:
data = load_data(number_of_items=48)

In [ ]:
temp=np.array(data['scalogram'])
print(temp.shape)

In [ ]:
encoder = LabelEncoder()
encoder.fit(data['label'])

In [ ]:
data['label'] = encoder.transform(data['label'])

In [ ]:
all_the_classes = encoder.classes_
mapping = {0: 'person_100',
 1: 'person_101',
 2: 'person_102',
 3: 'person_103',
 4: 'person_104',
 5: 'person_106',
 6: 'person_107',
 7: 'person_108',
 8: 'person_109',
 9: 'person_111',
 10: 'person_112',
 11: 'person_113',
 12: 'person_114',
 13: 'person_115',
 14: 'person_116',
 15: 'person_117',
 16: 'person_118',
 17: 'person_119',
 18: 'person_121',
 19: 'person_122',
 20: 'person_123',
 21: 'person_124',
 22: 'person_200',
 23: 'person_202',
 24: 'person_203',
 25: 'person_205',
 26: 'person_207',
 27: 'person_208',
 28: 'person_209',
 29: 'person_210',
 30: 'person_212',
 31: 'person_213',
 32: 'person_214',
 33: 'person_215',
 34: 'person_217',
 35: 'person_219',
 36: 'person_220',
 37: 'person_221',
 38: 'person_222',
 39: 'person_223',
 40: 'person_228',
 41: 'person_230',
 42: 'person_231',
 43: 'person_232',
 44: 'person_233',
 45: 'person_234'    
          }

#print(mapping)


# Get emotion from class number   
def get_name_from_class(class_number):
    """
        gets the corresponding subject from the class
    """
    if mapping.get(class_number,None):
        return mapping.get(class_number)
    else:
        return -1 # No such class

In [ ]:
# Extracting features and labels
features = np.array(data['scalogram'])
labels = data['label']

In [ ]:
print ("Shape of features: ", features.shape)
print ("Shape of labels: ", labels.shape)

In [ ]:
# Changing
labels = to_categorical(labels, num_classes=len(mapping))

In [ ]:
# Building model
model = Sequential()
model.add(Conv2D(32, kernel_size=5, input_shape=(224, 224, 1), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(32, kernel_size=5, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(150, activation="relu"))
#model.add(Dense(120, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(80, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(len(mapping), activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizers.RMSprop(),metrics=['accuracy'])

In [ ]:
# Splitting
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, shuffle=True, random_state=17)

In [ ]:
tensorboard = TensorBoard()
earlystopping = EarlyStopping(patience=3)

In [ ]:
# Scaling
features_train = features_train.astype("float32")
features_test = features_test.astype("float32")

In [ ]:
features_train = features_train / 1/255
features_test = features_test / 1/255

In [ ]:
features_train = features_train.reshape(len(features_train), 224, 224, 1)
features_test = features_test.reshape(len(features_test), 224, 224, 1)

In [ ]:
# Training
model.fit(features_train, labels_train, epochs=100, batch_size=16, callbacks=[tensorboard,earlystopping], validation_data=(features_test, labels_test))

In [ ]:
model.save("2DCNNButterworthExtended.h5") # Saving the model

In [ ]:
# Test an image
def test_image(image,label):
    #print(image.shape)
    prediction = np.argmax(model.predict(image.reshape(1,224,224,1)))
    result_predict=get_emotion_from_class(prediction)
    actual=np.argmax(label)
    result_actual=get_emotion_from_class(actual)
    #print(prediction)
    #print(actual)
    return result_actual,result_predict

    
# Get emotion from class number   
def get_emotion_from_class(class_number):
    """
        gets the corresponding label from the class
    """
    if mapping.get(class_number,None):
      return mapping.get(class_number)
    else:
      return -1 # No such class

In [ ]:
for i in range(len(features_test)):
    actual,predicted=test_image(features_test[i],labels_test[i])
    print("Model predicts: "+predicted+" and actual label is: "+actual)
    if(predicted!=actual):
        print("Wrong prediction")
    print("-------------------------------------------------------------")